In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [2]:
df_train = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
calendar = pd.read_csv('m5-forecasting-accuracy/calendar.csv')
price = pd.read_csv('m5-forecasting-accuracy/sell_prices.csv')
# df_test = pd.read_csv('m5-forecasting-accuracy/sample_submission.csv')

In [3]:
# # startpoints 찾아서 이전 데이터 지우기
startpoints = np.zeros(df_train.shape[0])
for idx in tqdm(range(df_train.shape[0])):
    startpoints[idx]= np.where(df_train.iloc[idx,6:].values>0)[0].min().astype(int)
start_dict = dict(zip(df_train['id'], startpoints))

100%|███████████████████████████████████████████████████████████████████████████| 30490/30490 [00:36<00:00, 841.66it/s]


In [4]:
drop_cols = []
cat_cols = []
drop_cols += ['date','d','id']
tr_last = 1913
# F_1~28 만들기  1914~1941 까지 
for i in range(tr_last+1, tr_last+1+28):   df_train['d_%s'%i] = 0

# # Unpivot
df_train = pd.melt(df_train, id_vars=df_train.columns[:6], value_vars=df_train.columns[6:],
       var_name = 'day', value_name = 'volume')



In [5]:
df_train = pd.merge(df_train, calendar, left_on = 'day', right_on ='d')
# snap 합치기
snap = np.zeros(df_train.shape[0])
snap[df_train[(df_train['state_id']=='CA')&(df_train['snap_CA']==1)].index] +=1
snap[df_train[(df_train['state_id']=='TX')&(df_train['snap_TX']==1)].index] +=1
snap[df_train[(df_train['state_id']=='WI')&(df_train['snap_WI']==1)].index] +=1
df_train['snap'] = snap
drop_cols += ['snap_CA','snap_TX','snap_WI']


cat_cols += [ 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
#               'wday', 'month', 'year', # 이게 크리티컬?
            'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap']


# ????

In [6]:
# Sell price
df_train.head()
df_train = pd.merge(df_train, price)

# # Start point 찾기 ::  0.1% 데이터를 날릴  수 있다. 

df_train['startpoint'] = df_train['id'].map(start_dict).astype(int)#.astype(str)
df_train['startpoints'] = df_train['day'].str.slice(start=2).astype(int) >=df_train['startpoint']
print(df_train['startpoints'].value_counts())
df_train = df_train[df_train['startpoints']]
print(df_train.shape)
df_train.drop(['startpoint','startpoints'],axis =1, inplace= True)

True     46816555
False       65122
Name: startpoints, dtype: int64
(46816555, 26)


In [7]:
%%time

# 왜 Shift를 해야할까?
# Shift 28을 하지 않으면 예측값이 뒤로 가면갈 수록  F1->F28 예측 할 수 있는 변수가 줄어든게 된다.
# 28일은 한달을 의미한다. 최근 한달간의 경향성을 보는 것으로 보면 되겠다.
# 28일을 56일로 늘리면 안되나? - 최근 한달간의 경향성이 반영이 안되는 것이낙?
# https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50

df_train['volume_7'] = df_train[['id','volume']].groupby("id")['volume'].shift(7)
df_train['volume_28'] = df_train[['id','volume']].groupby("id")['volume'].shift(28)

print("mean")

df_train['rmean_7_7'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_7_28'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_7_50'] = df_train[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

df_train['rmean_28_7'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
df_train['rmean_28_28'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
df_train['rmean_28_50'] = df_train[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())

# # print("std")
# # full_df['rstd_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).std())
# # full_df['rstd_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).std())
# # full_df['rstd_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).std())

# # print("max")
# # full_df['rmax_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).max())
# # full_df['rmax_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).max())
# # full_df['rmax_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).max())

# # print("min")
# # full_df['rmin_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7 ,min_periods=1).min())
# # full_df['rmin_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28 ,min_periods=1).min())
# # full_df['rmin_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(50 ,min_periods=1).min())

# # print("count")
# # full_df['rcnt_7'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(7).count() if x>0).fllna(0)
# # full_df['rcnt_28'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)
# # full_df['rcnt_50'] = full_df[['id','volume']].groupby("id")['volume'].transform(lambda x: x.rolling(28).count() if x>0).fllna(0)


mean
Wall time: 3min 4s


In [8]:
df_train['date'] =  pd.to_datetime(df_train["date"])

In [9]:
df_train['week'] = getattr(df_train["date"].dt, "weekofyear").astype("int16")
df_train['quarter'] = getattr(df_train["date"].dt,"quarter").astype("int16")
df_train['mday'] = getattr(df_train["date"].dt, "day").astype("int16")


In [10]:
cols =['event_name_1','event_type_1','event_name_2','event_type_2']
df_train[cols]= df_train[cols].fillna('NaN')
print(df_train.shape)


(46816555, 35)


In [11]:
df_train.dropna(inplace =True)

In [12]:
df_train.shape

(44468825, 35)

In [13]:
drop_cols += ["wm_yr_wk", "weekday"]  ## 이게 문제?
drop_cols

['date', 'd', 'id', 'snap_CA', 'snap_TX', 'snap_WI', 'wm_yr_wk', 'weekday']

In [14]:
df_train.head()
tr_last = 1913
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
train_id = df_train['id']
df_test_id = df_train[df_train['day'].isin(testday)]['id']

In [15]:
df_train= df_train.drop(drop_cols,axis =1 )
# df_test =df_test.drop(drop_cols,axis =1)

In [16]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,volume_28,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50,week,quarter,mday
1010975,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_78,0,1,4,2011,...,0.0,6.857143,1.785714,1.88,0.285714,1.285714,2.90,15,2,16
1010976,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_79,0,2,4,2011,...,0.0,4.000000,1.714286,1.68,0.000000,1.142857,2.66,15,2,17
1010977,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_80,0,3,4,2011,...,0.0,5.142857,2.000000,1.76,0.000000,1.000000,2.36,16,2,18
1010978,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_81,5,4,4,2011,...,0.0,5.714286,2.214286,1.80,0.000000,0.928571,2.36,16,2,19
1010979,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_82,23,5,4,2011,...,0.0,4.714286,2.428571,1.88,0.000000,0.714286,2.36,16,2,20


In [17]:
# Encoding
for col in tqdm(cat_cols) :  # encoding -1이 문제?
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col]).astype(np.int8)

    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:03<00:00,  6.38s/it]


In [18]:
# Test 분리하기
testday = ['d_%s'% x for x in range(tr_last+1, tr_last+1+28)]
df_test = df_train.copy()
df_train = df_train[~df_train['day'].isin(testday)]
# train_col = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day', 'volume', 'wday', 'month', 'year',
#  'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap', 'sell_price', 'volume_7', 'volume_28',
#  'rmean_7_7', 'rmean_7_28', 'rmean_7_50', 'rmean_28_7', 'rmean_28_28', 'rmean_28_50', 'week', 'quarter', 'mday']
# df_train = df_train.loc[:,train_col]

In [19]:
df_test.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,volume_28,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50,week,quarter,mday
1010975,-92,3,1,0,0,d_78,0,1,4,2011,...,0.0,6.857143,1.785714,1.88,0.285714,1.285714,2.90,15,2,16
1010976,-92,3,1,0,0,d_79,0,2,4,2011,...,0.0,4.000000,1.714286,1.68,0.000000,1.142857,2.66,15,2,17
1010977,-92,3,1,0,0,d_80,0,3,4,2011,...,0.0,5.142857,2.000000,1.76,0.000000,1.000000,2.36,16,2,18
1010978,-92,3,1,0,0,d_81,5,4,4,2011,...,0.0,5.714286,2.214286,1.80,0.000000,0.928571,2.36,16,2,19
1010979,-92,3,1,0,0,d_82,23,5,4,2011,...,0.0,4.714286,2.428571,1.88,0.000000,0.714286,2.36,16,2,20


In [20]:
# %%time

# from sklearn.model_selection import train_test_split

# x_train , x_valid = train_test_split(df_train, test_size =0.15, random_state = 99)
# y_train, y_valid = x_train['volume'], x_valid['volume']

# x_train = x_train.drop(['day','volume'], axis =1)
# x_valid = x_valid.drop(['day','volume'], axis =1)

In [21]:
df_train.head()

,item_id,dept_id,cat_id,store_id,state_id,day,volume,wday,month,year,...,volume_28,rmean_7_7,rmean_7_28,rmean_7_50,rmean_28_7,rmean_28_28,rmean_28_50,week,quarter,mday
1010975,-92,3,1,0,0,d_78,0,1,4,2011,...,0.0,6.857143,1.785714,1.88,0.285714,1.285714,2.90,15,2,16
1010976,-92,3,1,0,0,d_79,0,2,4,2011,...,0.0,4.000000,1.714286,1.68,0.000000,1.142857,2.66,15,2,17
1010977,-92,3,1,0,0,d_80,0,3,4,2011,...,0.0,5.142857,2.000000,1.76,0.000000,1.000000,2.36,16,2,18
1010978,-92,3,1,0,0,d_81,5,4,4,2011,...,0.0,5.714286,2.214286,1.80,0.000000,0.928571,2.36,16,2,19
1010979,-92,3,1,0,0,d_82,23,5,4,2011,...,0.0,4.714286,2.428571,1.88,0.000000,0.714286,2.36,16,2,20


In [22]:
# 마지막 28일 데이터 빼고 학습
cols=[f"d_{i}" for i in range(1913-28,1914)]
df_valid = df_train[df_train['day'].isin(cols)]
df_train = df_train[~df_train['day'].isin(cols)]

In [23]:
states = [999,243,498,45,32]

In [24]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 5000,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
        'n_jobs' :10 
}

In [25]:
%%time

folds = 3
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_log_error
import pickle

for state in states:
    
    seed = state
    skf = StratifiedKFold(n_splits=folds, random_state=seed, shuffle=True)

    for idx, (train_index, test_index) in enumerate(skf.split(df_train.index,df_train['volume'])):
        print(idx)
        x_train = df_train.iloc[train_index].drop(['day','volume'], axis =1)
        y_train = df_train.iloc[train_index]['volume']
        x_valid = df_train.iloc[test_index].drop(['day','volume'], axis =1)
        y_valid = df_train.iloc[test_index]['volume'] 

        # Modeling
        lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=cat_cols)
        lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=cat_cols)
        gbm = lgb.train(params, lgb_train,
    #                     num_boost_round=1000, 
                        valid_sets=(lgb_train, lgb_eval),
                        early_stopping_rounds= 50,#100,
                        verbose_eval=50) #100)

        pickle.dump(gbm,open( "20200505_model_%s_best_seed%s.pkl"%(idx,seed), "wb" ))

        del gbm
        gc.collect()


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


0


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.59317	valid_1's rmse: 2.62731
[100]	training's rmse: 2.5079	valid_1's rmse: 2.54336
[150]	training's rmse: 2.48386	valid_1's rmse: 2.52345
[200]	training's rmse: 2.46293	valid_1's rmse: 2.50677
[250]	training's rmse: 2.44412	valid_1's rmse: 2.49258
[300]	training's rmse: 2.42672	valid_1's rmse: 2.47989
[350]	training's rmse: 2.40967	valid_1's rmse: 2.46834
[400]	training's rmse: 2.39722	valid_1's rmse: 2.46067
[450]	training's rmse: 2.38676	valid_1's rmse: 2.45517
[500]	training's rmse: 2.37641	valid_1's rmse: 2.44957
[550]	training's rmse: 2.36641	valid_1's rmse: 2.4444
[600]	training's rmse: 2.3582	valid_1's rmse: 2.44006
[650]	training's rmse: 2.35078	valid_1's rmse: 2.43665
[700]	training's rmse: 2.34326	valid_1's rmse: 2.43341
[750]	training's rmse: 2.33641	valid_1's rmse: 2.43059
[800]	training's rmse: 2.33008	valid_1's rmse: 2.42786
[850]	training's rmse: 2.32284	valid_1's rmse: 2.4248
[900]	tra

[2250]	training's rmse: 2.21243	valid_1's rmse: 2.37037
[2300]	training's rmse: 2.20985	valid_1's rmse: 2.36984
[2350]	training's rmse: 2.20758	valid_1's rmse: 2.36934
[2400]	training's rmse: 2.20465	valid_1's rmse: 2.36881
[2450]	training's rmse: 2.20185	valid_1's rmse: 2.36827
[2500]	training's rmse: 2.19953	valid_1's rmse: 2.36758
[2550]	training's rmse: 2.19716	valid_1's rmse: 2.36714
[2600]	training's rmse: 2.19441	valid_1's rmse: 2.36664
[2650]	training's rmse: 2.19238	valid_1's rmse: 2.36598
[2700]	training's rmse: 2.18928	valid_1's rmse: 2.36554
[2750]	training's rmse: 2.18654	valid_1's rmse: 2.36509
[2800]	training's rmse: 2.18374	valid_1's rmse: 2.3646
[2850]	training's rmse: 2.18119	valid_1's rmse: 2.36403
[2900]	training's rmse: 2.17908	valid_1's rmse: 2.36371
Early stopping, best iteration is:
[2888]	training's rmse: 2.17955	valid_1's rmse: 2.36359
2
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.61761	valid_1's rmse: 2.63063
[100]	tra

...

0


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.57171	valid_1's rmse: 2.58249
[100]	training's rmse: 2.50376	valid_1's rmse: 2.52083
[150]	training's rmse: 2.4817	valid_1's rmse: 2.50369
[200]	training's rmse: 2.45948	valid_1's rmse: 2.48731
[250]	training's rmse: 2.44251	valid_1's rmse: 2.47552
[300]	training's rmse: 2.42777	valid_1's rmse: 2.46577
[350]	training's rmse: 2.41259	valid_1's rmse: 2.45582
[400]	training's rmse: 2.39919	valid_1's rmse: 2.44816
[450]	training's rmse: 2.38758	valid_1's rmse: 2.44178
[500]	training's rmse: 2.37835	valid_1's rmse: 2.43723
[550]	training's rmse: 2.36856	valid_1's rmse: 2.43226
[600]	training's rmse: 2.36134	valid_1's rmse: 2.4291
[650]	training's rmse: 2.35384	valid_1's rmse: 2.42591
[700]	training's rmse: 2.34597	valid_1's rmse: 2.42273
[750]	training's rmse: 2.33969	valid_1's rmse: 2.42003
[800]	training's rmse: 2.33349	valid_1's rmse: 2.4174
[850]	training's rmse: 2.327	valid_1's rmse: 2.41521
[900]	trai

[2250]	training's rmse: 2.2103	valid_1's rmse: 2.39368
[2300]	training's rmse: 2.20698	valid_1's rmse: 2.39275
[2350]	training's rmse: 2.20386	valid_1's rmse: 2.39204
[2400]	training's rmse: 2.20041	valid_1's rmse: 2.39156
[2450]	training's rmse: 2.19747	valid_1's rmse: 2.39109
[2500]	training's rmse: 2.19425	valid_1's rmse: 2.39029
[2550]	training's rmse: 2.19115	valid_1's rmse: 2.38928
[2600]	training's rmse: 2.18865	valid_1's rmse: 2.38862
[2650]	training's rmse: 2.18596	valid_1's rmse: 2.38795
[2700]	training's rmse: 2.18306	valid_1's rmse: 2.38757
[2750]	training's rmse: 2.18001	valid_1's rmse: 2.38692
[2800]	training's rmse: 2.17716	valid_1's rmse: 2.38639
[2850]	training's rmse: 2.17471	valid_1's rmse: 2.38589
[2900]	training's rmse: 2.17218	valid_1's rmse: 2.38544
[2950]	training's rmse: 2.16988	valid_1's rmse: 2.38486
[3000]	training's rmse: 2.16736	valid_1's rmse: 2.38453
[3050]	training's rmse: 2.16501	valid_1's rmse: 2.38414
[3100]	training's rmse: 2.16238	valid_1's rmse: 2

[4500]	training's rmse: 2.1112	valid_1's rmse: 2.359
[4550]	training's rmse: 2.10953	valid_1's rmse: 2.35863
[4600]	training's rmse: 2.10781	valid_1's rmse: 2.35834
[4650]	training's rmse: 2.1061	valid_1's rmse: 2.35791
[4700]	training's rmse: 2.10421	valid_1's rmse: 2.3575
[4750]	training's rmse: 2.10242	valid_1's rmse: 2.35727
[4800]	training's rmse: 2.10064	valid_1's rmse: 2.35696
[4850]	training's rmse: 2.09898	valid_1's rmse: 2.35658
[4900]	training's rmse: 2.09717	valid_1's rmse: 2.35627
[4950]	training's rmse: 2.09557	valid_1's rmse: 2.35599
[5000]	training's rmse: 2.09372	valid_1's rmse: 2.35583
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 2.09372	valid_1's rmse: 2.35583
Wall time: 16h 59min 26s


In [28]:
%%time
df_test['id'] = train_id
max_lag  = 120
trn_lst = 1913
for state in states:
    print(state)
    if state == 999: continue
    #df_test 값이 Reset이 안되어 있다 !!!!!  - df test를 유지하는 것으로 하자
    tmp_test = df_test.copy() # 추가 !!
    for tdelta in range(1, 29):
        f_day = trn_lst+tdelta
        print(f_day)
        days = [f"d_{i}" for i in range(trn_lst-max_lag+tdelta,f_day+1)]
        tst = tmp_test[tmp_test['day'].isin(days)]

        print("rolling")
        tst['volume_7'] = tst[['id','volume']].groupby("id")['volume'].shift(7)
        tst['volume_28'] = tst[['id','volume']].groupby("id")['volume'].shift(28)

        print("mean")
        tst['rmean_7_7'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(7).mean())
        tst['rmean_7_28'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(28).mean())
        tst['rmean_7_50'] = tst[['id','volume_7']].groupby("id")['volume_7'].transform(lambda x: x.rolling(50).mean())

        tst['rmean_28_7'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(7).mean())
        tst['rmean_28_28'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(28).mean())
        tst['rmean_28_50'] = tst[['id','volume_28']].groupby("id")['volume_28'].transform(lambda x: x.rolling(50).mean())


        tst = tst[tst['day'] == "d_%s"%(f_day)]
        t_id,t_volume,t_day = tst['id'],tst['volume'],tst['day']
        tst = tst.drop(['id','volume','day'], axis =1)

        # Crossvalidation 
        for idx in range(folds):
            gbm = pickle.load(open( "20200505_model_%s_best_seed%s.pkl"%(idx,state), "rb" ))
            tmp_test.loc[tmp_test.day=="d_%s"%(f_day),'volume'] += 1.028*gbm.predict(tst) / folds
    
    cols = [f"d_{i}" for i in range(1914,1942)]

    sub = tmp_test[tmp_test['day'].isin(cols)].loc[:,['id','volume']]
    sub['F']= [f"F{rank}" for rank in sub.groupby("id")["id"].cumcount()+1]
    sub = sub.set_index(["id", "F" ]).unstack()["volume"].reset_index()
    sub.sort_values("id", inplace = True)
    sub.reset_index(drop=True, inplace = True)                                                   
    sub =sub[['id']+["F%s"% x for x in range(1,29)]]

    sub = sub.fillna(0)

    sub2 = sub.copy()
    sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
    sub = pd.concat([sub, sub2], axis=0, sort=False)
    sub.to_csv("submission_20200506_0_seed%s.csv"%state,index=False)
    
    del tmp_test
    gc.collect()


999
243
1914
rolling


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


mean


C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\yseon\Anaconda3\envs\M5\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

1915
rolling
mean
1916
rolling
mean
1917
rolling
mean
1918
rolling
mean
1919
rolling
mean
1920
rolling
mean
1921
rolling
mean
1922
rolling
mean
1923
rolling
mean
1924
rolling
mean
1925
rolling
mean
1926
rolling
mean
1927
rolling
mean
1928
rolling
mean
1929
rolling
mean
1930
rolling
mean
1931
rolling
mean
1932
rolling
mean
1933
rolling
mean
1934
rolling
mean
1935
rolling
mean
1936
rolling
mean
1937
rolling
mean
1938
rolling
mean
1939
rolling
mean
1940
rolling
mean
1941
rolling
mean
498
1914
rolling
mean
1915
rolling
mean
1916
rolling
mean
1917
rolling
mean
1918
rolling
mean
1919
rolling
mean
1920
rolling
mean
1921
rolling
mean
1922
rolling
mean
1923
rolling
mean
1924
rolling
mean
1925
rolling
mean
1926
rolling
mean
1927
rolling
mean
1928
rolling
mean
1929
rolling
mean
1930
rolling
mean
1931
rolling
mean
1932
rolling
mean
1933
rolling
mean
1934
rolling
mean
1935
rolling
mean
1936
rolling
mean
1937
rolling
mean
1938
rolling
mean
1939
rolling
mean
1940
rolling
mean
1941
rolling
mean
45
191

In [27]:
# cols = [f"d_{i}" for i in range(1914,1942)]

# sub = df_test[df_test['day'].isin(cols)].loc[:,['id','volume']]
# sub['F']= [f"F{rank}" for rank in sub.groupby("id")["id"].cumcount()+1]
# sub = sub.set_index(["id", "F" ]).unstack()["volume"].reset_index()
# sub.sort_values("id", inplace = True)
# sub.reset_index(drop=True, inplace = True)                                                   
# sub =sub[['id']+["F%s"% x for x in range(1,29)]]

# sub = sub.fillna(0)

# sub2 = sub.copy()
# sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
# sub = pd.concat([sub, sub2], axis=0, sort=False)
# sub.to_csv("submission_20200505_0.csv",index=False)

In [28]:
sub.isnull().sum()

F
id     0
F1     0
F2     0
F3     0
F4     0
F5     0
F6     0
F7     0
F8     0
F9     0
F10    0
F11    0
F12    0
F13    0
F14    0
F15    0
F16    0
F17    0
F18    0
F19    0
F20    0
F21    0
F22    0
F23    0
F24    0
F25    0
F26    0
F27    0
F28    0
dtype: int64

In [29]:
from typing import Union

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm


class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame, tst):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()
#         train_id = train_id
        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices
        self.tst = tst
        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        scale = np.where(scale != 0 , scale, 1)
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)
            all_scores.append(lv_scores.sum())

        return np.mean(all_scores)
    
    
class WRMSSEForLightGBM(WRMSSEEvaluator):

    def feval(self, preds, dtrain):
#         print(preds.shape, self.tst.shape)
#         tst= self.df[self.df['day'].isin(valid_target_columns)]
#         tst['id'] = train_id.loc[tst.index]
        tmp = self.tst.copy()
        tmp['preds'] = preds
        tmp=  tmp.set_index(['id',"day"]).unstack()["preds"].reset_index()
        tmp =  tmp.fillna(0)

        val = pd.DataFrame()
        val['id'] = self.train_df['id']
        pred = pd.merge(val, tmp, how = 'left')
        pred = pred.fillna(0)
#         print(pred.columns)
#         print(self.valid_target_columns)
        pred = pred.loc[:,self.valid_target_columns ]
#         cv_scores.append(evaluator.score(pred))
#         preds = preds.reshape(self.valid_df[self.valid_target_columns].shape)
        score = self.score(pred)
        return 'WRMSSE', score, False

In [30]:
# 마지막 28일 데이터로 검증
cv_scores =[]
cols = ["d_%s"%x for x in range(1,1914)]
days = list(df_train['day'].unique()) # 날짜 섞기
cols = [x for x in cols if x in days]

d_valid=[f"d_{i}" for i in range(1913-28,1914)]
d_train = [x for x in cols if x not in d_valid]
tmp = pd.read_csv('m5-forecasting-accuracy/sales_train_validation.csv')
tmp_train = pd.concat([tmp.iloc[:,:6],tmp.loc[:, d_train]], axis =1)
tmp_valid = tmp.loc[:, d_valid]
#     evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price)
tst= df_valid[df_valid['day'].isin(d_valid)]
tst['id'] = train_id.loc[tst.index]


evaluator = WRMSSEEvaluator(tmp_train, tmp_valid, calendar, price,tst)
for state in states:
    print(state)
    for idx in range(folds):

        # gbm = pickle.dump(gbm,open( "20200502_model_%s_r1_5000.pkl"%idx, "wb" ))
        gbm = pickle.load(open( "20200505_model_%s_best_seed%s.pkl"%(idx,state), "rb" ))
        preds= gbm.predict(df_valid.drop(['day','volume'], axis =1))
        tst= df_valid[df_valid['day'].isin(d_valid)]
        tst['id'] = train_id.loc[tst.index]
        tst['preds'] = preds
        tst= tst.set_index(['id',"day"]).unstack()["preds"].reset_index()
        tst = tst.fillna(0)

        val = pd.DataFrame()
        val['id'] = tmp_train['id']
        pred = pd.merge(val,tst, how = 'left')
        pred = pred.fillna(0)
        pred = pred.loc[:,d_valid]
        cv_scores.append(evaluator.score(pred))
        print(state, idx,"WRMSSE :", evaluator.score(pred))
    print(state, "CV score" , np.mean(cv_scores))


999
999 0 WRMSSE : 0.5308819776047743
999 1 WRMSSE : 0.5182642052738963
999 2 WRMSSE : 0.5358917656293912
999 CV score 0.5283459828360205
243
243 0 WRMSSE : 0.5339033635461353
243 1 WRMSSE : 0.5383453261087875
243 2 WRMSSE : 0.5250614624912074
243 CV score 0.530391350109032
498
498 0 WRMSSE : 0.5249241083925535
498 1 WRMSSE : 0.5381342798502133
498 2 WRMSSE : 0.5370496208798273
498 CV score 0.5313840121974207
45
45 0 WRMSSE : 0.531560943707681
45 1 WRMSSE : 0.5402675316632751
45 2 WRMSSE : 0.5360487959897174
45 CV score 0.532527781761455
32
32 0 WRMSSE : 0.5329068294848568
32 1 WRMSSE : 0.5359323958815846
32 2 WRMSSE : 0.5299491452827523
32 CV score 0.5326081167857769
